In [2]:
import numpy as np
import os
from sklearn.cluster import MiniBatchKMeans
import pandas as pd
import _pickle as pkl
from tqdm import tqdm
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics.pairwise import chi2_kernel
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import GradientBoostingClassifier

In [3]:
'''
 MODE -> 'surf' or 'cnn' for the respective features to be used 
 EVENTS -> List of events to train for
 POOL -> Pooling method for keyframes in a video -> 'max' or 'mean'
 MODEL_NAME -> Filename prefix to save the trained models
'''

MODE = "surf"
EVENTS = ['P001','P002','P003']
POOL = 'mean'
MODEL_NAME = "CNN_RF"
CNN_DIR = "cnn"
surf_directory = "surf"

# All videos
all_videos = open("list/all.video").readlines()
all_videos = [i.strip() for i in all_videos]

# Train videos
train_videos = open("list/train.video").readlines()
train_videos = [i.strip() for i in train_videos]

# VAlidation videos
val_videos = open("list/val.video").readlines()
val_videos = [i.strip() for i in val_videos]

# Test videos
test_videos = open("../all_test.video").readlines()
test_videos = [i.strip() for i in test_videos]


# train labels
train_labels = {}
with open("../all_trn.lst") as f:
    for line in f.readlines():
        file, label = line.strip().split()
        train_labels[file] = label
        
# val labels
val_labels = {}
with open("../all_val.lst") as f:
    for line in f.readlines():
        file, label = line.strip().split()
        val_labels[file] = label

# Create feature vectors of keyframes

In [4]:
if MODE == 'surf':
    train_features = []
    for video in tqdm(all_videos[:]):
        file_path = os.path.join(surf_directory,video+".feat")
        with open(file_path,"rb") as f:
            a = pkl.load(f,encoding='bytes')
            train_features.extend(a)

    # Delete None entries
    mark = []
    for i,j in enumerate(train_features):
        if type(train_features[i]) != np.ndarray:
            mark.append(i)
    train_features = [train_features[i] for i in range(len(train_features)) if i not in mark]

    # Stack into a numpy array
    train_features = np.vstack(train_features)

else:
    train_features = []
    for video in tqdm(all_videos[:]):
        file_path = os.path.join(CNN_DIR,video+".feat")
        with open(file_path,"rb") as f:
            a = pkl.load(f,encoding='bytes')
            for item in a:
                train_features.extend(item)

    # Delete None entries
    mark = []
    for i,j in enumerate(train_features):
        if type(train_features[i]) != np.ndarray:
            mark.append(i)
    train_features = [train_features[i] for i in range(len(train_features)) if i not in mark]

    # Stack into a numpy array
    train_features = np.vstack(train_features)

100%|██████████| 2935/2935 [00:52<00:00, 72.45it/s] 


# Train k-means


In [6]:
N_CLUSTERS = 800
BATCH_SIZE = 128

kmeans = MiniBatchKMeans(n_clusters=N_CLUSTERS,batch_size=BATCH_SIZE)
kmeans.fit(train_features)


/home/ubuntu/anaconda3/lib/python3.7/site-packages/sklearn/cluster/k_means_.py:1559: RuntimeWarning: init_size=384 should be larger than k=800. Setting it to 3*k
  init_size=init_size)
/home/ubuntu/anaconda3/lib/python3.7/site-packages/sklearn/cluster/k_means_.py:1559: RuntimeWarning: init_size=384 should be larger than k=800. Setting it to 3*k
  init_size=init_size)
/home/ubuntu/anaconda3/lib/python3.7/site-packages/sklearn/cluster/k_means_.py:1559: RuntimeWarning: init_size=384 should be larger than k=800. Setting it to 3*k
  init_size=init_size)


MiniBatchKMeans(batch_size=128, compute_labels=True, init='k-means++',
        init_size=None, max_iter=100, max_no_improvement=10,
        n_clusters=800, n_init=3, random_state=None,
        reassignment_ratio=0.01, tol=0.0, verbose=0)

# Convert keyframes into bag-of-words features

In [7]:
if MODE == "cnn":
    # Input video directory
    DIR = CNN_DIR
    # Directory with the generated feature vectors
    FEATURE_DIR = "kmeans_cnn"
    # Directory to save trained models
    MODEL_DIR = "models"
    # Directory to save generated prediction files
    PRED_DIR = "cnn_pred"
else:
    # Input video directory
    DIR = surf_directory
    # Directory with the generated feature vectors
    FEATURE_DIR = "kmeans"
    # Directory to save trained models
    MODEL_DIR = "models"
    # Directory to save generated prediction files
    PRED_DIR = "surf_pred"

In [8]:

    
for video in tqdm(all_videos[:]):
    file_path = os.path.join(DIR,video+".feat")
    with open(file_path,"rb") as f:
        keyframes = pkl.load(f,encoding='bytes')
        video_surf = []
        for keyframe in keyframes:
            if type(keyframe) == np.ndarray:
                # Predict clusters for each vector in the keyframe
                keyframe_preds = kmeans.predict(keyframe)
                # Create one-hot encoded vector of cluster assignments
                keyframe_kmeans = np.zeros((len(keyframe_preds),N_CLUSTERS))
                for i,clus in enumerate(keyframe_preds):
                    keyframe_kmeans[i][clus] = 1
                # Reduce one-hot encoded vector into a histogram for each keyframe
                keyframe_kmeans = np.sum(keyframe_kmeans,keepdims=True,axis=0)
                # Append bag-of-words repr into an array
                video_surf.append(keyframe_kmeans)
                
            # None features -> zero array
            else:
                n = np.zeros((1,N_CLUSTERS))
                video_surf.append(n)
                
        # Convert video surf into numpy array
        if len(video_surf) > 0:
            video_surf = np.vstack(video_surf)
            if POOL == 'max':
                video_surf = np.amax(video_surf,axis=0,keepdims=True)
            else:
                video_surf = np.mean(video_surf,axis=0,keepdims=True)
        else:
            video_surf = np.zeros((1,N_CLUSTERS))
            
        output_dir = os.path.join(FEATURE_DIR,video+".feat")
        with open(output_dir,"wb") as o:
            pkl.dump(video_surf,o)

100%|██████████| 2935/2935 [01:45<00:00, 27.94it/s]


# Train Classifiers

In [19]:
MODEL_USED = 'rf'
MODEL_NAME = MODE + "_" + MODEL_USED

# Function to load the SURF video features for a given dataset
def load_data(feature_dir,video_list,labels_map=None):
    feats = []
    labels = []
    # load features for train set
    for video in tqdm(video_list[:]):
        feat_path = os.path.join(FEATURE_DIR,video+".feat")
        with open(feat_path,"rb") as f:
            video_surf = pkl.load(f,encoding='bytes')
            if labels_map:
                if labels_map[video] != 'NULL':
                    labels.append(labels_map[video])
                    feats.append(video_surf)
                
    feats = np.vstack(feats)
    if labels_map:
        return feats,np.array(labels)
    else:
        return feats,None

train_x,train_y = load_data(FEATURE_DIR,train_videos,train_labels)
val_x,val_y = load_data(FEATURE_DIR,val_videos,val_labels)

100%|██████████| 400/400 [00:00<00:00, 31954.17it/s]


In [20]:
# Combine train and validation set into a single train dataset
combine = True
if combine:
    all_train_x = np.vstack([train_x,val_x])
    all_train_y = np.concatenate((train_y,val_y))

    # Shuffle dataset
    idxs = [i for i in range(all_train_x.shape[0])]
    np.random.shuffle(idxs)

    all_train_x = all_train_x[idxs]
    all_train_y = all_train_y[idxs]
else:
    all_train_x = train_x
    all_train_y = train_y

In [31]:
# Train 3 separate models for each event
# MLP parameters
hidden_layers = (1024,2048)
lr = 1e-3
batch_size = 250
alpha = 1e-5
loss = []


# Random Forest params
num_estimators = 500
max_depth = 20

for event in tqdm(EVENTS[:]):
    # Train for event
    event_y = (all_train_y==event).astype('int')
    if MODEL_USED == 'rf':
        model = RandomForestClassifier(n_estimators=num_estimators,max_depth=max_depth)
#         model = GradientBoostingClassifier(n_estimators=num_estimators,max_depth=max_depth)
    elif MODEL_USED == 'mlp':
        model = MLPClassifier(hidden_layer_sizes=hidden_layers,
                         alpha=alpha,
                         batch_size=batch_size,
                         learning_rate_init=lr,
                         max_iter=2000,
                            verbose=False,
                         )
    model.fit(all_train_x,event_y)
    if MODEL_USED== 'mlp':
        loss.append(model.loss_)
        print("Event {} loss : {} ".format(event,model.loss_))
    # Save model
    model_path = os.path.join(MODEL_DIR,MODEL_NAME+"_" +event+".model")
    with open(model_path,"wb") as o:
        pkl.dump(model,o)



  0%|          | 0/3 [00:00<?, ?it/s]

 33%|███▎      | 1/3 [00:00<00:01,  1.67it/s]

 67%|██████▋   | 2/3 [00:01<00:00,  1.67it/s]

100%|██████████| 3/3 [00:01<00:00,  1.68it/s]



# Generate Validation and Test files

In [32]:
for event in EVENTS[:]:
    # Load model for event and predict
    model_path = os.path.join(MODEL_DIR,MODEL_NAME+"_" +event+".model")
    with open(model_path,"rb") as l:
        model = pkl.load(l,encoding='bytes')
        val_pred = os.path.join(PRED_DIR,MODE+"_val_"+event+".csv")
        test_pred = os.path.join(PRED_DIR,MODE+"_"+event+".csv")
        
        with open(test_pred,"w") as o: 
            for video in tqdm(test_videos):
                feat_path = os.path.join(FEATURE_DIR,video+".feat")
                with open(feat_path,"rb") as f:
                    feat  = pkl.load(f,encoding='bytes')

                    prediction = model.predict_proba(feat)
                    o.write("{}\n".format(prediction[0][1]))
#                     prediction = model.decision_function(feat)
#                     o.write("{}\n".format(prediction[0]))


                    
#         with open(val_pred,"w") as o: 
#             for video in tqdm(val_videos):
#                 feat_path = os.path.join(FEATURE_DIR,video+".feat")
#                 with open(feat_path,"rb") as f:
#                     feat  = pkl.load(f,encoding='bytes')
#                     prediction = model.predict_proba(feat)
#                     o.write("{}\n".format(prediction[0][1]))
# #                     prediction = model.decision_function(feat)
# #                     o.write("{}\n".format(prediction[0]))
                     
                    
p1 = open(PRED_DIR + "/" + MODE + "_val_P001.csv").readlines()
p2 = open(PRED_DIR + "/" + MODE + "_val_P002.csv").readlines()
p3 = open(PRED_DIR + "/" + MODE + "_val_P003.csv").readlines()

p1 = [float(p.strip()) for p in p1]
p2 = [float(p.strip()) for p in p2]
p3 = [float(p.strip()) for p in p3]

acc = [0,0,0]
total = [0,0,0]

THRESHOLD = 0
correct = 0
with open("../all_val.lst") as f:
    lines = f.readlines()
    for idx,line in enumerate(tqdm(lines)):
        filename, truth = line.strip().split()
        label = [p1[idx],p2[idx],p3[idx]]
        pred = np.argmax(label)
        if truth != "NULL":
            if EVENTS[pred] == truth:
                acc[pred] += 1
            total[EVENTS.index(truth)] += 1
            
acc = np.array(acc)/np.array(total)
print("Event P001 val accuracy : {0:1.2f}\nEvent P002 val accuracy : {1:1.2f}\nEvent P003 val accuracy : {2:1.2f}\n"
      .format(acc[0],acc[1],acc[2]))

# Write params in log
with open("logs.txt","a") as log:
    if MODEL_USED == 'mlp':
        log.write("{0}#{1}#{2}#{3}#{4}#{5:0.2f}#{6:0.2f}#{7:0.2f}#{8:0.2f}#{9:0.2f}#{10:0.2f}\n".format(MODEL_USED,hidden_layers,lr,batch_size,
                                                       alpha,loss[0],loss[1],loss[2],acc[0],acc[1],acc[2]))
    elif MODEL_USED == 'rf':
        log.write("{0}#{1}#{2}#{3:0.2f}#{4:0.2f}#{5:0.2f}\n".format(MODEL_USED,num_estimators,max_depth,acc[0],acc[1],acc[2]))




  0%|          | 0/1699 [00:00<?, ?it/s]

  0%|          | 3/1699 [00:00<01:01, 27.46it/s]

  0%|          | 6/1699 [00:00<01:03, 26.58it/s]

  1%|          | 9/1699 [00:00<01:04, 26.24it/s]

  1%|          | 12/1699 [00:00<01:05, 25.59it/s]

  1%|          | 15/1699 [00:00<01:05, 25.69it/s]

  1%|          | 18/1699 [00:00<01:03, 26.32it/s]

  1%|          | 21/1699 [00:00<01:04, 25.87it/s]

  1%|▏         | 24/1699 [00:00<01:03, 26.36it/s]

  2%|▏         | 27/1699 [00:01<01:05, 25.65it/s]

  2%|▏         | 30/1699 [00:01<01:05, 25.47it/s]

  2%|▏         | 33/1699 [00:01<01:04, 25.64it/s]

  2%|▏         | 36/1699 [00:01<01:03, 26.11it/s]

  2%|▏         | 39/1699 [00:01<01:04, 25.67it/s]

  2%|▏         | 42/1699 [00:01<01:03, 26.04it/s]

  3%|▎         | 45/1699 [00:01<01:02, 26.39it/s]

  3%|▎         | 48/1699 [00:01<01:03, 26.11it/s]

  3%|▎         | 51/1699 [00:01<01:03, 25.84it/s]

  3%|▎         | 54/1699 [00:02<01:04, 25.53it/s]

  3%|▎         | 57/1699 [00:02<01:04, 25

 25%|██▌       | 426/1699 [00:16<00:47, 26.63it/s]

 25%|██▌       | 429/1699 [00:16<00:47, 26.71it/s]

 25%|██▌       | 432/1699 [00:16<00:47, 26.66it/s]

 26%|██▌       | 435/1699 [00:16<00:47, 26.42it/s]

 26%|██▌       | 438/1699 [00:17<00:47, 26.65it/s]

 26%|██▌       | 441/1699 [00:17<00:48, 25.99it/s]

 26%|██▌       | 444/1699 [00:17<00:48, 26.11it/s]

 26%|██▋       | 447/1699 [00:17<00:47, 26.37it/s]

 26%|██▋       | 450/1699 [00:17<00:48, 25.91it/s]

 27%|██▋       | 453/1699 [00:17<00:47, 26.07it/s]

 27%|██▋       | 456/1699 [00:17<00:49, 25.20it/s]

 27%|██▋       | 459/1699 [00:17<00:49, 25.30it/s]

 27%|██▋       | 462/1699 [00:18<00:48, 25.70it/s]

 27%|██▋       | 465/1699 [00:18<00:47, 25.85it/s]

 28%|██▊       | 468/1699 [00:18<00:48, 25.32it/s]

 28%|██▊       | 471/1699 [00:18<00:48, 25.32it/s]

 28%|██▊       | 474/1699 [00:18<00:47, 25.58it/s]

 28%|██▊       | 477/1699 [00:18<00:47, 25.99it/s]

 28%|██▊       | 480/1699 [00:18<00:46, 26.17it/s]

 28%|██▊    

 50%|████▉     | 849/1699 [00:33<00:32, 26.37it/s]

 50%|█████     | 852/1699 [00:33<00:31, 26.62it/s]

 50%|█████     | 855/1699 [00:33<00:31, 27.18it/s]

 51%|█████     | 858/1699 [00:33<00:33, 25.02it/s]

 51%|█████     | 861/1699 [00:33<00:32, 26.05it/s]

 51%|█████     | 864/1699 [00:33<00:31, 26.71it/s]

 51%|█████     | 867/1699 [00:33<00:31, 26.25it/s]

 51%|█████     | 870/1699 [00:33<00:31, 26.63it/s]

 51%|█████▏    | 873/1699 [00:34<00:30, 26.78it/s]

 52%|█████▏    | 876/1699 [00:34<00:30, 27.09it/s]

 52%|█████▏    | 879/1699 [00:34<00:31, 26.20it/s]

 52%|█████▏    | 882/1699 [00:34<00:31, 25.98it/s]

 52%|█████▏    | 885/1699 [00:34<00:31, 26.18it/s]

 52%|█████▏    | 888/1699 [00:34<00:30, 26.38it/s]

 52%|█████▏    | 891/1699 [00:34<00:31, 26.00it/s]

 53%|█████▎    | 894/1699 [00:34<00:31, 25.90it/s]

 53%|█████▎    | 897/1699 [00:34<00:31, 25.22it/s]

 53%|█████▎    | 900/1699 [00:35<00:31, 25.16it/s]

 53%|█████▎    | 903/1699 [00:35<00:31, 25.56it/s]

 53%|█████▎ 

 75%|███████▍  | 1266/1699 [00:49<00:17, 24.85it/s]

 75%|███████▍  | 1269/1699 [00:49<00:17, 25.02it/s]

 75%|███████▍  | 1272/1699 [00:49<00:16, 25.18it/s]

 75%|███████▌  | 1275/1699 [00:49<00:16, 25.10it/s]

 75%|███████▌  | 1278/1699 [00:49<00:16, 25.35it/s]

 75%|███████▌  | 1281/1699 [00:49<00:16, 25.28it/s]

 76%|███████▌  | 1284/1699 [00:49<00:16, 25.32it/s]

 76%|███████▌  | 1287/1699 [00:50<00:16, 25.20it/s]

 76%|███████▌  | 1290/1699 [00:50<00:16, 25.13it/s]

 76%|███████▌  | 1293/1699 [00:50<00:15, 25.48it/s]

 76%|███████▋  | 1296/1699 [00:50<00:15, 25.85it/s]

 76%|███████▋  | 1299/1699 [00:50<00:15, 25.67it/s]

 77%|███████▋  | 1302/1699 [00:50<00:15, 26.08it/s]

 77%|███████▋  | 1305/1699 [00:50<00:16, 23.67it/s]

 77%|███████▋  | 1308/1699 [00:50<00:15, 24.45it/s]

 77%|███████▋  | 1311/1699 [00:50<00:15, 24.63it/s]

 77%|███████▋  | 1314/1699 [00:51<00:15, 25.17it/s]

 78%|███████▊  | 1317/1699 [00:51<00:15, 25.08it/s]

 78%|███████▊  | 1320/1699 [00:51<00:15, 25.14

 99%|█████████▉| 1680/1699 [01:05<00:00, 25.54it/s]

 99%|█████████▉| 1683/1699 [01:05<00:00, 25.92it/s]

 99%|█████████▉| 1686/1699 [01:05<00:00, 25.65it/s]

 99%|█████████▉| 1689/1699 [01:05<00:00, 25.84it/s]

100%|█████████▉| 1692/1699 [01:05<00:00, 26.17it/s]

100%|█████████▉| 1695/1699 [01:05<00:00, 25.64it/s]

100%|█████████▉| 1698/1699 [01:06<00:00, 26.27it/s]

100%|██████████| 1699/1699 [01:06<00:00, 25.72it/s]

  0%|          | 0/1699 [00:00<?, ?it/s]

  0%|          | 3/1699 [00:00<01:02, 27.00it/s]

  0%|          | 6/1699 [00:00<01:03, 26.71it/s]

  1%|          | 9/1699 [00:00<01:04, 26.14it/s]

  1%|          | 12/1699 [00:00<01:05, 25.77it/s]

  1%|          | 15/1699 [00:00<01:04, 25.96it/s]

  1%|          | 18/1699 [00:00<01:05, 25.77it/s]

  1%|          | 21/1699 [00:00<01:04, 25.91it/s]

  1%|▏         | 24/1699 [00:00<01:04, 25.86it/s]

  2%|▏         | 27/1699 [00:01<01:06, 25.27it/s]

  2%|▏         | 30/1699 [00:01<01:04, 25.80it/s]

  2%|▏         | 33/1699 [0

...

 22%|██▏       | 372/1699 [00:14<00:50, 26.02it/s]

 22%|██▏       | 375/1699 [00:14<00:51, 25.77it/s]

 22%|██▏       | 378/1699 [00:14<00:50, 25.99it/s]

 22%|██▏       | 381/1699 [00:14<00:51, 25.52it/s]

 23%|██▎       | 384/1699 [00:14<00:51, 25.57it/s]

 23%|██▎       | 387/1699 [00:14<00:51, 25.65it/s]

 23%|██▎       | 390/1699 [00:15<00:51, 25.66it/s]

 23%|██▎       | 393/1699 [00:15<00:51, 25.31it/s]

 23%|██▎       | 396/1699 [00:15<00:50, 25.59it/s]

 23%|██▎       | 399/1699 [00:15<00:55, 23.55it/s]

 24%|██▎       | 402/1699 [00:15<00:53, 24.35it/s]

 24%|██▍       | 405/1699 [00:15<00:53, 24.34it/s]

 24%|██▍       | 408/1699 [00:15<00:51, 24.90it/s]

 24%|██▍       | 411/1699 [00:15<00:49, 25.79it/s]

 24%|██▍       | 414/1699 [00:16<00:48, 26.46it/s]

 25%|██▍       | 417/1699 [00:16<00:47, 26.97it/s]

 25%|██▍       | 420/1699 [00:16<00:49, 25.86it/s]

 25%|██▍       | 423/1699 [00:16<00:48, 26.33it/s]

 25%|██▌       | 426/1699 [00:16<00:48, 26.24it/s]

 25%|██▌    

 47%|████▋     | 795/1699 [00:30<00:35, 25.62it/s]

 47%|████▋     | 798/1699 [00:30<00:35, 25.56it/s]

 47%|████▋     | 801/1699 [00:30<00:34, 25.82it/s]

 47%|████▋     | 804/1699 [00:31<00:33, 26.35it/s]

 47%|████▋     | 807/1699 [00:31<00:33, 26.78it/s]

 48%|████▊     | 810/1699 [00:31<00:33, 26.50it/s]

 48%|████▊     | 813/1699 [00:31<00:33, 26.46it/s]

 48%|████▊     | 816/1699 [00:31<00:33, 26.08it/s]

 48%|████▊     | 819/1699 [00:31<00:34, 25.63it/s]

 48%|████▊     | 822/1699 [00:31<00:33, 26.33it/s]

 49%|████▊     | 825/1699 [00:31<00:33, 25.81it/s]

 49%|████▊     | 828/1699 [00:31<00:33, 26.07it/s]

 49%|████▉     | 831/1699 [00:32<00:33, 25.77it/s]

 49%|████▉     | 834/1699 [00:32<00:32, 26.41it/s]

 49%|████▉     | 837/1699 [00:32<00:33, 26.06it/s]

 49%|████▉     | 840/1699 [00:32<00:32, 26.12it/s]

 50%|████▉     | 843/1699 [00:32<00:32, 26.34it/s]

 50%|████▉     | 846/1699 [00:32<00:35, 23.77it/s]

 50%|████▉     | 849/1699 [00:32<00:34, 24.62it/s]

 50%|█████  

 72%|███████▏  | 1215/1699 [00:46<00:18, 26.58it/s]

 72%|███████▏  | 1218/1699 [00:47<00:18, 26.00it/s]

 72%|███████▏  | 1221/1699 [00:47<00:18, 26.36it/s]

 72%|███████▏  | 1224/1699 [00:47<00:17, 26.80it/s]

 72%|███████▏  | 1227/1699 [00:47<00:17, 27.25it/s]

 72%|███████▏  | 1230/1699 [00:47<00:17, 27.03it/s]

 73%|███████▎  | 1233/1699 [00:47<00:17, 26.21it/s]

 73%|███████▎  | 1236/1699 [00:47<00:18, 25.60it/s]

 73%|███████▎  | 1239/1699 [00:47<00:17, 26.11it/s]

 73%|███████▎  | 1242/1699 [00:47<00:17, 26.20it/s]

 73%|███████▎  | 1245/1699 [00:48<00:17, 26.08it/s]

 73%|███████▎  | 1248/1699 [00:48<00:17, 25.68it/s]

 74%|███████▎  | 1251/1699 [00:48<00:17, 25.65it/s]

 74%|███████▍  | 1254/1699 [00:48<00:17, 25.19it/s]

 74%|███████▍  | 1257/1699 [00:48<00:17, 25.72it/s]

 74%|███████▍  | 1260/1699 [00:48<00:17, 25.38it/s]

 74%|███████▍  | 1263/1699 [00:48<00:16, 25.68it/s]

 75%|███████▍  | 1266/1699 [00:48<00:16, 25.73it/s]

 75%|███████▍  | 1269/1699 [00:49<00:16, 25.56

 96%|█████████▌| 1629/1699 [01:03<00:02, 25.74it/s]

 96%|█████████▌| 1632/1699 [01:03<00:02, 25.79it/s]

 96%|█████████▌| 1635/1699 [01:03<00:02, 25.55it/s]

 96%|█████████▋| 1638/1699 [01:03<00:02, 25.68it/s]

 97%|█████████▋| 1641/1699 [01:03<00:02, 25.63it/s]

 97%|█████████▋| 1644/1699 [01:03<00:02, 25.61it/s]

 97%|█████████▋| 1647/1699 [01:03<00:02, 25.83it/s]

 97%|█████████▋| 1650/1699 [01:03<00:01, 26.15it/s]

 97%|█████████▋| 1653/1699 [01:04<00:01, 26.80it/s]

 97%|█████████▋| 1656/1699 [01:04<00:01, 26.87it/s]

 98%|█████████▊| 1659/1699 [01:04<00:01, 26.51it/s]

 98%|█████████▊| 1662/1699 [01:04<00:01, 26.33it/s]

 98%|█████████▊| 1665/1699 [01:04<00:01, 26.25it/s]

 98%|█████████▊| 1668/1699 [01:04<00:01, 26.25it/s]

 98%|█████████▊| 1671/1699 [01:04<00:01, 25.61it/s]

 99%|█████████▊| 1674/1699 [01:04<00:00, 25.60it/s]

 99%|█████████▊| 1677/1699 [01:04<00:00, 25.26it/s]

 99%|█████████▉| 1680/1699 [01:05<00:00, 24.99it/s]

 99%|█████████▉| 1683/1699 [01:05<00:00, 25.10

Event P001 val accuracy : 1.00
Event P002 val accuracy : 0.89
Event P003 val accuracy : 0.78



# Generate Kaggle Submission

In [25]:
p1 = open(PRED_DIR + "/" + MODE + "_P001.csv").readlines()
p2 = open(PRED_DIR + "/" + MODE + "_P002.csv").readlines()
p3 = open(PRED_DIR + "/" + MODE + "_P003.csv").readlines()

p1 = [float(p.strip()) for p in p1]
p2 = [float(p.strip()) for p in p2]
p3 = [float(p.strip()) for p in p3]

THRESHOLD = 0
with open((MODE+"_kaggle_prediction.csv"),"w") as o:
    o.write("VideoID,Label\n")
    for idx,video in enumerate(tqdm(test_videos)):
        label = [p1[idx],p2[idx],p3[idx]]
        idx = np.argmax(label)
        
        if label[idx] > THRESHOLD:
            pred = idx +1
        else:
            pred = 0
            
        o.write("{},{}\n".format(video,pred))
        


100%|██████████| 1699/1699 [00:00<00:00, 89181.32it/s]